## Algorithmic Trading: Mean Reversion 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import datetime

import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression


### Import Libraries

### Get Data

In [ ]:
# Load data from CSV
csv_file_path = 'USA30IDXUSD.csv'
csv_file_path4h = 'USA30IDXUSD4h.csv'
df_ltf = pd.read_csv(csv_file_path)
df_htf = pd.read_csv(csv_file_path4h)

Format HTF Data

In [ ]:
# Combine Date and Timestamp columns into a single datetime column
df_htf['date'] = pd.to_datetime(df_htf['Date'].astype(str) + ' ' + df_htf['Timestamp'].astype(str))

# Drop the original Date and Timestamp columns if no longer needed
df_htf.drop(['Date', 'Timestamp'], axis=1, inplace=True)

# Rename columns
df_htf = df_htf.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
})

# Reverse index
df_htf = df_htf.reindex(index=df_htf.index[::-1])
df_htf.reset_index(level=0, inplace=True)

# Drop the 'index' column if it's not needed
df_htf.drop('index', axis=1, inplace=True)


format LTF Data

In [ ]:
# Combine Date and Timestamp columns into a single datetime column
df_ltf['date'] = pd.to_datetime(df_ltf['Date'].astype(str) + ' ' + df_ltf['Timestamp'].astype(str))

# Drop the original Date and Timestamp columns if no longer needed
df_ltf.drop(['Date', 'Timestamp'], axis=1, inplace=True)

# Rename columns
df_ltf = df_ltf.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
})

# Reverse index
df_ltf = df_ltf.reindex(index=df_ltf.index[::-1])
df_ltf.reset_index(level=0, inplace=True)

# Drop the 'index' column if it's not needed
df_ltf.drop('index', axis=1, inplace=True)

Select Start and End Time

In [ ]:
df_ltf = df_ltf[((df_ltf['date'] >= pd.to_datetime('2024-01-01')) & (df_ltf['date'] <= pd.to_datetime('2024-08-05')))]
df_htf = df_htf[((df_htf['date'] >= pd.to_datetime('2024-01-01')) & (df_htf['date'] <= pd.to_datetime('2024-08-07')))]

### Calculate ma

In [ ]:
#Short moving averages (5-20 periods) are best suited for short-term trends and trading
df_ltf['ma_200'] = df_ltf['close'].rolling(200).mean()
df_htf['ema_100'] = df_htf['close'].ewm(span=100, adjust=False).mean()

In [ ]:
#plot for validation
plt.figure(figsize=(12,5))
plt.xticks(rotation=45)

plt.plot(df_ltf['date'], df_ltf['close'], label = 'LTF Close')
plt.plot(df_ltf['date'], df_ltf['ma_200'], label = '200 ma')

plt.legend()
plt.show()


Calculate the Angle of the Linear regression 

In [ ]:
def calculate_ma_angle(df, ma_column='ema_100', window=10):
    """
    Calculate the angle of the moving average line over a rolling window.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the moving average column.
    - ma_column (str): Name of the column containing the moving average values.
    - window (int): Number of data points to include in each rolling window.

    Returns:
    - List of angles in degrees with NaNs for the initial rows without enough data.
    """
    angles = np.full(len(df), np.nan)  # Initialize with NaNs

    for i in range(len(df) - window + 1):
        # Select 'window' data points
        subset = df.iloc[i:i + window]
        
        # Prepare data for calculating the slope of the moving average
        X = np.arange(window).reshape(-1, 1)  # Time steps (0, 1, 2, ..., window-1)
        y = subset[ma_column].values  # Moving average values

        # Fit linear regression model on the entire window
        model_window = LinearRegression().fit(X, y)
        slope_window = model_window.coef_[0]  # Slope of the regression line for the entire window

        # Calculate angle in degrees
        angle_rad = np.arctan(slope_window)
        angle_deg = np.degrees(angle_rad)
        angles[i + window - 1] = angle_deg  # Assign angle to the last point of the window

    return angles


In [ ]:

df_htf['ma_angle'] = calculate_ma_angle(df_htf, ma_column='ema_100')

# Plot for validation
plt.figure(figsize=(12, 5))
plt.xticks(rotation=45)

# Plotting the close price and 100-period moving average
plt.plot(df_htf['date'], df_htf['close'], label='Close')
plt.plot(df_htf['date'], df_htf['ema_100'], label='100-period EMA')

# Fill background based on angle
for i in range(len(df_htf) - 1):
    angle = df_htf['ma_angle'].iloc[i]
    if angle > 50:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightcoral', alpha=0.5)  # Bullish: Light Green
    elif angle < -50:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightgreen', alpha=0.5)  # Bearish: Light Coral
    else:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='darkgrey', alpha=0.5)  # Ranging: Dark Grey

# Annotate the angle values
for i in range(0, len(df_htf), max(1, len(df_htf) // 10)):  # Adjust the interval as needed
    plt.annotate(f'{df_htf["ma_angle"].iloc[i]:.2f}', 
                 (df_htf['date'].iloc[i], df_htf['close'].iloc[i]), 
                 textcoords="offset points", 
                 xytext=(0,10), 
                 ha='center', 
                 fontsize=8, 
                 color='black')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Close Price and 100-period EMA with Angle Annotations')
plt.legend()
plt.show()

### Calculate RSI

In [ ]:
def gain(value):
    if value < 0:
        return 0
    else:
        return value

In [ ]:
def loss(value):
    if value > 0:
        return 0
    else:
        return abs(value)

In [ ]:

period = 13

#Calculate price delta
df_ltf['delta'] = df_ltf['close'].diff()

#Classify delta into gain & loss
df_ltf['gain'] = df_ltf['delta'].apply(lambda x:gain(x))
df_ltf['loss'] = df_ltf['delta'].apply(lambda x:loss(x))

#Calculate ema 
df_ltf['ema_gain'] = df_ltf['gain'].ewm(period).mean()
df_ltf['ema_loss'] = df_ltf['loss'].ewm(period).mean()

#Calculate RSI
df_ltf['rs'] = df_ltf['ema_gain']/df_ltf['ema_loss']
df_ltf['rsi'] = df_ltf['rs'].apply(lambda x: 100 - (100/(x+1)))

In [ ]:
# Ensure 'date' is a column and not the index
if 'date' in df_ltf.index.names:
    df_ltf.reset_index(inplace=True)

# Plot RSI for validation
plt.figure(figsize=(12, 5))
plt.xticks(rotation=45)

x_axis = df_ltf['date']

x_axis = df_ltf['date']

plt.plot(x_axis, df_ltf['rsi'])
plt.axhline(30, c= (.5,.5,.5), ls='--')
plt.axhline(70, c= (.5,.5,.5), ls='--')

plt.title('RSI (13)', fontweight="bold")
plt.ylim([0, 100])

### Implementing buy/sell

#### Rules:
    1. Buy when 13-period RSI below 30 (buy next day) & Price below lower bollinger band
    2. Sell when 10-period RSI above 70 (sell next day) & Price above upper bollinger band

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Initialize signal column
df_ltf['signal'] = 0

# Merge LTF and HTF DataFrames on datetime
df_combined = pd.merge(df_ltf, df_htf[['date', 'ma_angle']], on='date', how='left')

# Forward fill the ma_angle to propagate the 4-hour trend to 15-minute intervals
df_combined['ma_angle'] = df_combined['ma_angle'].ffill()

# Define conditions for buy and sell signals
buy_condition = (df_combined['rsi'] < 30) & (df_combined['ma_angle'] < -50)
sell_condition = (df_combined['rsi'] > 70) & (df_combined['ma_angle'] > 50)

# Identify initial buy and sell signals
df_combined.loc[buy_condition, 'signal'] = 1
df_combined.loc[sell_condition, 'signal'] = -1

# Enforce the 4-candle row difference between signals
last_signal_index = -4  # Initialize to -4 to allow the first signal

for i in range(len(df_combined)):
    if df_combined.loc[i, 'signal'] != 0:
        if i - last_signal_index >= 120:  # Check if 4 or more rows have passed
            last_signal_index = i  # Update the index of the last valid signal
        else:
            df_combined.loc[i, 'signal'] = 0  # Invalidate the signal if within 4 rows

# Shift signals to the next trading day (optional step depending on your strategy)
df_combined['signal'] = df_combined['signal'].shift()
df_combined['signal'] = df_combined['signal'].fillna(0)

# Prepare the data for plotting
plt.figure(figsize=(16, 10))

# Plot the close price and moving average on the first subplot
plt.subplot(3, 1, 1)
plt.plot(df_combined['date'], df_combined['close'], label='Close Price', color='blue')

# Plot buy and sell signals on the close price chart
plt.scatter(df_combined['date'][df_combined['signal'] == 1], df_combined['close'][df_combined['signal'] == 1], marker='^', color='g', label='Buy Signal', s=100, edgecolor='black', linewidth=1.5)
plt.scatter(df_combined['date'][df_combined['signal'] == -1], df_combined['close'][df_combined['signal'] == -1], marker='v', color='r', label='Sell Signal', s=100, edgecolor='black', linewidth=1.5)

plt.title('Close Price with Buy and Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Plot the RSI on the second subplot
plt.subplot(3, 1, 2)
plt.plot(df_combined['date'], df_combined['rsi'], label='RSI (13)', color='purple')

# Plot buy and sell signals on the RSI chart at specific levels
plt.scatter(df_combined['date'][df_combined['signal'] == 1], df_combined['rsi'][df_combined['signal'] == 1], marker='^', color='g', label='Buy Signal', s=100, edgecolor='black', linewidth=1.5)
plt.scatter(df_combined['date'][df_combined['signal'] == -1], df_combined['rsi'][df_combined['signal'] == -1], marker='v', color='r', label='Sell Signal', s=100, edgecolor='black', linewidth=1.5)

plt.axhline(30, color='grey', linestyle='--')
plt.axhline(70, color='grey', linestyle='--')

plt.title('RSI (13) with Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim([0, 100])
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Plot the HTF EMA and MA angle on the third subplot
plt.subplot(3, 1, 3)
plt.plot(df_htf['date'], df_htf['close'], label='HTF Close Price', color='blue')
plt.plot(df_htf['date'], df_htf['ema_100'], label='100 EMA', color='orange')

for i in range(len(df_htf) - 1):
    angle = df_htf['ma_angle'].iloc[i]
    if angle < -50:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightgreen', alpha=0.3)  # Bearish: Light Coral
    elif angle > 50:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightcoral', alpha=0.3)  # Bullish: Light Green
    else:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='darkgrey', alpha=0.3)  # Neutral: Dark Grey

plt.title('HTF Close Price and 100 EMA with MA Angles')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


### Backtesting Strategy

In [ ]:
#print(df_combined[['date', 'close', 'signal', 'buy_date', 'sell_date']].head(40))
import back_testing as bt
print(df_combined.head())
bt.backtest_with_drawdown_and_equity_curve(df_combined)